In [0]:
import pandas as pd
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import os
from sklearn.model_selection import train_test_split
import shutil #pip install pytest-shutil
from PIL import Image
import PIL
import imageio
import matplotlib.pyplot as plt
import glob

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Create new directory for the images
base_dir = 'base_dir'
os.mkdir(base_dir)

In [0]:
# Training file directory
train_dir = os.path.join(base_dir, 'train_dir')
os.mkdir(train_dir)

# Validation file directory
val_dir = os.path.join(base_dir, 'val_dir')
os.mkdir(val_dir)

# Testing file directory
test_dir = os.path.join(base_dir, 'test_dir')
os.mkdir(test_dir)

In [0]:
# Create new folders in training directory for each of the 7 classes

nv = os.path.join(train_dir, 'nv')
os.mkdir(nv)
mel = os.path.join(train_dir, 'mel')
os.mkdir(mel)
bkl = os.path.join(train_dir, 'bkl')
os.mkdir(bkl)
bcc = os.path.join(train_dir, 'bcc')
os.mkdir(bcc)
akiec = os.path.join(train_dir, 'akiec')
os.mkdir(akiec)
vasc = os.path.join(train_dir, 'vasc')
os.mkdir(vasc)
df = os.path.join(train_dir, 'df')
os.mkdir(df)

# Create new folders in validation directory for each of the 7 classes

nv = os.path.join(val_dir, 'nv')
os.mkdir(nv)
mel = os.path.join(val_dir, 'mel')
os.mkdir(mel)
bkl = os.path.join(val_dir, 'bkl')
os.mkdir(bkl)
bcc = os.path.join(val_dir, 'bcc')
os.mkdir(bcc)
akiec = os.path.join(val_dir, 'akiec')
os.mkdir(akiec)
vasc = os.path.join(val_dir, 'vasc')
os.mkdir(vasc)
df = os.path.join(val_dir, 'df')
os.mkdir(df)

# Create new folders in test directory for each of the 7 classes

nv = os.path.join(test_dir, 'nv')
os.mkdir(nv)
mel = os.path.join(test_dir, 'mel')
os.mkdir(mel)
bkl = os.path.join(test_dir, 'bkl')
os.mkdir(bkl)
bcc = os.path.join(test_dir, 'bcc')
os.mkdir(bcc)
akiec = os.path.join(test_dir, 'akiec')
os.mkdir(akiec)
vasc = os.path.join(test_dir, 'vasc')
os.mkdir(vasc)
df = os.path.join(test_dir, 'df')
os.mkdir(df)

In [0]:
# Read metadata file
mdf = pd.read_csv('../content/gdrive/My Drive/HAM10000_metadata.csv')

mdf.sample(5)

,lesion_id,image_id,dx,dx_type,age,sex,localization
3988,HAM_0005946,ISIC_0026368,nv,follow_up,55.0,male,genital
9691,HAM_0000673,ISIC_0029659,akiec,histo,70.0,female,face
5353,HAM_0005687,ISIC_0027880,nv,follow_up,65.0,male,upper extremity
6593,HAM_0003061,ISIC_0032262,nv,follow_up,65.0,female,upper extremity
705,HAM_0004630,ISIC_0026576,bkl,histo,85.0,male,face


In [0]:
# Set y as the data labels (column dx)
y = mdf['dx']

In [0]:
# Split into training, validation, and test
val_size = 0.1
test_size = 0.1
mdf_train_temp, mdf_remain, y_train_temp, y_remain = train_test_split(mdf, y, test_size = (0.2), random_state = 101)
new_test_size = np.around(test_size / (val_size + test_size),2)
mdf_val, mdf_test, y_val, y_test = train_test_split(mdf_remain, y_remain, test_size = new_test_size, random_state = 101)
mdf_train, mdf_discard, y_train, y_discard = train_test_split(mdf_train_temp, y_train_temp, test_size = 0.5, random_state = 101)
#print(mdf_train_temp.shape)
print(mdf_val.shape)
print(mdf_test.shape)
print(mdf_train.shape)

(1001, 7)
(1002, 7)
(4006, 7)


In [0]:
# Transfer the images into folders, set the image id as the index
mdf.set_index('image_id', inplace = True)

# Get a list of the images in each of the two folders
folder_1 = os.listdir('../content/gdrive/My Drive/ham10000_images_part_1')
folder_2 = os.listdir('../content/gdrive/My Drive/ham10000_images_part_2')

In [0]:
# Get a list of the train and validation images
train_list = list(mdf_train['image_id'])
val_list = list(mdf_val['image_id'])
test_list = list(mdf_test['image_id'])

In [0]:
akiec = 0
bcc = 0
bkl = 0
df = 0
mel = 0
nv = 0
vasc = 0
for image in train_list:
  r=mdf.loc[image, 'dx']
  if r == 'akiec':
    akiec = akiec + 1
  if r == 'bcc':
    bcc = bcc + 1
  if r == 'bkl':
    bkl = bkl + 1
  if r == 'df':
    df = df + 1
  if r == 'mel':
    mel = mel + 1
  if r == 'nv':
    nv = nv + 1
  if r == 'vasc':
    vasc = vasc + 1
print('akiec =', akiec)
print('bcc =', bcc)
print('bkl =', bkl)
print('df =', df)
print('mel =', mel)
print('nv =', nv)
print('vasc =', vasc)

akiec = 138
bcc = 195
bkl = 464
df = 46
mel = 429
nv = 2669
vasc = 65


In [0]:
# Transfer the training images

for image in train_list:
    
    fname = image + '.jpg'
    label = mdf.loc[image, 'dx']
    
    if fname in folder_1:
        # Source path to image
        src = os.path.join('../content/gdrive/My Drive/ham10000_images_part_1', fname)
        # Destination path to image
        dst = os.path.join(train_dir, label, fname)
        # Copy the image from the source to the destination
        shutil.copyfile(src, dst)

    if fname in folder_2:
        # Source path to image
        src = os.path.join('../content/gdrive/My Drive/ham10000_images_part_2', fname)
        # Destination path to image
        dst = os.path.join(train_dir, label, fname)
        # Copy the image from the source to the destination
        shutil.copyfile(src, dst)

In [0]:
# Transfer the validation images

for image in val_list:

    fname = image + '.jpg'
    label = mdf.loc[image, 'dx']
    
    if fname in folder_1:
        # Source path to image
        src = os.path.join('../content/gdrive/My Drive/ham10000_images_part_1', fname)
        # Destination path to image
        dst = os.path.join(val_dir, label, fname)
        # Copy the image from the source to the destination
        shutil.copyfile(src, dst)

    if fname in folder_2:
        # Source path to image
        src = os.path.join('../content/gdrive/My Drive/ham10000_images_part_2', fname)
        # Destination path to image
        dst = os.path.join(val_dir, label, fname)
        # Copy the image from the source to the destination
        shutil.copyfile(src, dst)

In [0]:
# Transfer the testing images

for image in test_list:

    fname = image + '.jpg'
    label = mdf.loc[image, 'dx']

    if fname in folder_1:
        # Source path to image
        src = os.path.join('../content/gdrive/My Drive/ham10000_images_part_1', fname)
        # Destination path to image
        dst = os.path.join(test_dir, label, fname)
        # Copy the image from the source to the destination
        shutil.copyfile(src, dst)

    if fname in folder_2:
        # Source path to image
        src = os.path.join('../content/gdrive/My Drive/ham10000_images_part_2', fname)
        # Destination path to image
        dst = os.path.join(test_dir, label, fname)
        # Copy the image from the source to the destination
        shutil.copyfile(src, dst)

In [0]:
# Check how many training images are in each folder

print(len(os.listdir('base_dir/train_dir/nv')))
print(len(os.listdir('base_dir/train_dir/mel')))
print(len(os.listdir('base_dir/train_dir/bkl')))
print(len(os.listdir('base_dir/train_dir/bcc')))
print(len(os.listdir('base_dir/train_dir/akiec')))
print(len(os.listdir('base_dir/train_dir/vasc')))
print(len(os.listdir('base_dir/train_dir/df')))

# Check how many validation images are in each folder

print(len(os.listdir('base_dir/val_dir/nv')))
print(len(os.listdir('base_dir/val_dir/mel')))
print(len(os.listdir('base_dir/val_dir/bkl')))
print(len(os.listdir('base_dir/val_dir/bcc')))
print(len(os.listdir('base_dir/val_dir/akiec')))
print(len(os.listdir('base_dir/val_dir/vasc')))
print(len(os.listdir('base_dir/val_dir/df')))

# Check how many test images are in each folder

print(len(os.listdir('base_dir/test_dir/nv')))
print(len(os.listdir('base_dir/test_dir/mel')))
print(len(os.listdir('base_dir/test_dir/bkl')))
print(len(os.listdir('base_dir/test_dir/bcc')))
print(len(os.listdir('base_dir/test_dir/akiec')))
print(len(os.listdir('base_dir/test_dir/vasc')))
print(len(os.listdir('base_dir/test_dir/df')))

2669
429
464
195
138
65
46
679
106
107
60
28
13
8
657
136
111
49
25
12
12


In [0]:
# custom rotation function by increments of 90 degrees
import random
def myFunc(image):
    image = np.array(image)
    return np.rot90(image,random.randint(1,4))

# Augment the data in the training set to 4000 images

class_list = ['nv', 'mel', 'bkl', 'bcc', 'akiec', 'vasc', 'df']

for item in class_list:

    # Create a temporary directory for the augmented images
    aug_dir = 'aug_dir'
    os.mkdir(aug_dir)

    # Create a directory within the base dir to store images of the same class
    img_dir = os.path.join(aug_dir, 'img_dir')
    os.mkdir(img_dir)

    # Choose a class
    img_class = item

    # List all the images in the directory
    img_list = os.listdir('base_dir/train_dir/' + img_class)

    # Copy images from the class train dir to the img_dir
    for fname in img_list:
        # Source path to image
        src = os.path.join('base_dir/train_dir/' + img_class, fname)
        # Destination path to image
        dst = os.path.join(img_dir, fname)
        # Copy the image from the source to the destination
        shutil.copyfile(src, dst)

    # Point to a dir containing the images and not to the images themselves
    path = aug_dir
    save_path = 'base_dir/train_dir/' + img_class

    # Create a data generator to augment the images in real time
    datagen = ImageDataGenerator(
        preprocessing_function=myFunc,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,
        brightness_range=(0.9,1.1))

    batch_size = 5

    aug_datagen = datagen.flow_from_directory(path,
                                              save_to_dir=save_path,
                                              save_prefix=fname[:-4],
                                              save_format='jpg',
                                              target_size=(224, 224),
                                              batch_size=batch_size)

    # Generate the augmented images and add them to the training folders
    num_aug_images_wanted = 4000  # total number of images we want to have in each class
    num_files = len(os.listdir(img_dir))
    num_batches = int(np.ceil((num_aug_images_wanted - num_files) / batch_size))

    # Run the generator and create about 6000 augmented images
    for i in range(0, num_batches):
        imgs, labels = next(aug_datagen)

    # Delete temporary directory with the raw image files
    shutil.rmtree('aug_dir')

Found 2669 images belonging to 1 classes.
Found 429 images belonging to 1 classes.
Found 464 images belonging to 1 classes.
Found 195 images belonging to 1 classes.
Found 138 images belonging to 1 classes.
Found 65 images belonging to 1 classes.
Found 46 images belonging to 1 classes.


In [0]:
import random
def myFunc(image):
    image = np.array(image)
    return np.rot90(image,random.randint(1,4))

# Augment the data in the val set to 800 images

class_list = ['nv', 'mel', 'bkl', 'bcc', 'akiec', 'vasc', 'df']

for item in class_list:

    # Create a temporary directory for the augmented images
    aug_dir = 'aug_dir'
    os.mkdir(aug_dir)

    # Create a directory within the base dir to store images of the same class
    img_dir = os.path.join(aug_dir, 'img_dir')
    os.mkdir(img_dir)

    # Choose a class
    img_class = item

    # List all the images in the directory
    img_list = os.listdir('base_dir/val_dir/' + img_class)

    # Copy images from the class train dir to the img_dir
    for fname in img_list:
        # Source path to image
        src = os.path.join('base_dir/val_dir/' + img_class, fname)
        # Destination path to image
        dst = os.path.join(img_dir, fname)
        # Copy the image from the source to the destination
        shutil.copyfile(src, dst)

    # Point to a dir containing the images and not to the images themselves
    path = aug_dir
    save_path = 'base_dir/val_dir/' + img_class

    # Create a data generator to augment the images in real time
    datagen = ImageDataGenerator(
        preprocessing_function=myFunc,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,
        brightness_range=(0.9,1.1))

    batch_size = 5

    aug_datagen = datagen.flow_from_directory(path,
                                              save_to_dir=save_path,
                                              save_prefix=fname[:-4],
                                              save_format='jpg',
                                              target_size=(224, 224),
                                              batch_size=batch_size)

    # Generate the augmented images and add them to the training folders
    num_aug_images_wanted = 800  # total number of images we want to have in each class
    num_files = len(os.listdir(img_dir))
    num_batches = int(np.ceil((num_aug_images_wanted - num_files) / batch_size))

    # Run the generator and create about 6000 augmented images
    for i in range(0, num_batches):
        imgs, labels = next(aug_datagen)

    # Delete temporary directory with the raw image files
    shutil.rmtree('aug_dir')

Found 679 images belonging to 1 classes.
Found 106 images belonging to 1 classes.
Found 107 images belonging to 1 classes.
Found 60 images belonging to 1 classes.
Found 28 images belonging to 1 classes.
Found 13 images belonging to 1 classes.
Found 8 images belonging to 1 classes.


In [0]:
# Resizing
class_list = ['mel', 'bkl', 'bcc', 'akiec', 'vasc', 'df','nv']
cwd = os.getcwd()
for item in class_list:
    for fname in os.listdir(os.path.join(cwd, 'base_dir', 'train_dir', item)):
        if len(fname) == 16:
            img = Image.open(os.path.join(os.path.join(cwd, 'base_dir', 'train_dir', item), fname))
            img = img.resize((224,224))
            img.save(os.path.join(os.path.join(cwd, 'base_dir', 'train_dir', item), fname), 'JPEG')
                     
for item in class_list:
    for fname in os.listdir(os.path.join(cwd, 'base_dir', 'val_dir', item)):
        if len(fname) == 16:
            img = Image.open(os.path.join(os.path.join(cwd, 'base_dir', 'val_dir', item), fname))
            img = img.resize((224,224))
            img.save(os.path.join(os.path.join(cwd, 'base_dir', 'val_dir', item), fname), 'JPEG')

for item in class_list:
    for fname in os.listdir(os.path.join(cwd, 'base_dir', 'test_dir', item)):
        if len(fname) == 16:
            img = Image.open(os.path.join(os.path.join(cwd, 'base_dir', 'test_dir', item), fname))
            img = img.resize((224,224))
            img.save(os.path.join(os.path.join(cwd, 'base_dir', 'test_dir', item), fname), 'JPEG')

In [0]:
# new folders and transfer
train_all = os.path.join(train_dir, 'train_all')
os.mkdir(train_all)

dst_dir = "base_dir/train_dir/train_all"
for item in class_list: 
    src_dir = "base_dir/train_dir/" + item
    for jpgfile in glob.iglob(os.path.join(src_dir, "*.jpg")):
        shutil.copy(jpgfile, dst_dir)

val_all = os.path.join(val_dir, 'val_all')
os.mkdir(val_all)

dst_dir = "base_dir/val_dir/val_all"
for item in class_list: 
    src_dir = "base_dir/val_dir/" + item
    for jpgfile in glob.iglob(os.path.join(src_dir, "*.jpg")):
        shutil.copy(jpgfile, dst_dir)

test_all = os.path.join(test_dir, 'test_all')
os.mkdir(test_all)

dst_dir = "base_dir/test_dir/test_all"
for item in class_list: 
    src_dir = "base_dir/test_dir/" + item
    for jpgfile in glob.iglob(os.path.join(src_dir, "*.jpg")):
        shutil.copy(jpgfile, dst_dir)

In [0]:
total_num_im = len(os.listdir(os.path.join(cwd, 'base_dir', 'train_dir', 'train_all')))
X_train = np.ndarray(shape = (total_num_im,224,224,3), dtype = "uint8")
for idx, fname in enumerate(os.listdir(os.path.join(cwd, 'base_dir', 'train_dir', 'train_all'))):
    I = np.asarray(PIL.Image.open("base_dir/train_dir/train_all/"+fname))
    X_train[idx,:,:,:] = I
print('X_train:',X_train.shape) # validate correct shape

total_num_im = len(os.listdir(os.path.join(cwd, 'base_dir', 'val_dir', 'val_all')))
X_val = np.ndarray(shape = (total_num_im,224,224,3), dtype = "uint8")
for idx, fname in enumerate(os.listdir(os.path.join(cwd, 'base_dir', 'val_dir', 'val_all'))):
    I = np.asarray(PIL.Image.open("base_dir/val_dir/val_all/"+fname))
    X_val[idx,:,:,:] = I
print('X_val:',X_val.shape) # validate correct shape

total_num_im = len(os.listdir(os.path.join(cwd, 'base_dir', 'test_dir', 'test_all')))
X_test = np.ndarray(shape = (total_num_im,224,224,3), dtype = "uint8")
for idx, fname in enumerate(os.listdir(os.path.join(cwd, 'base_dir', 'test_dir', 'test_all'))):
    I = np.asarray(PIL.Image.open("base_dir/test_dir/test_all/"+fname))
    X_test[idx,:,:,:] = I
print('X_test:',X_test.shape) # validate correct shape

X_train: (27631, 224, 224, 3)
X_val: (5262, 224, 224, 3)
X_test: (1002, 224, 224, 3)


In [0]:
Y_train = []
for idx, fname in enumerate(os.listdir(os.path.join(cwd, 'base_dir', 'train_dir', 'train_all'))):
    label = mdf.loc[fname[0:12], 'dx']
    Y_train.append(label)
Y_train = pd.get_dummies(Y_train)
Y_train = Y_train.values # convert to numpy array
print('Y_train:',Y_train.shape) # validate correct shape
# col1 col2 col3 col4 col5 col6 col7
# akiec bcc  bkl  df   mel  nv   vasc

Y_val = []
for idx, fname in enumerate(os.listdir(os.path.join(cwd, 'base_dir', 'val_dir', 'val_all'))):
    label = mdf.loc[fname[0:12], 'dx']
    Y_val.append(label)
Y_val = pd.get_dummies(Y_val)
Y_val = Y_val.values
print('Y_val:',Y_val.shape) # validate correct shape

Y_test = []
for idx, fname in enumerate(os.listdir(os.path.join(cwd, 'base_dir', 'test_dir', 'test_all'))):
    label = mdf.loc[fname[0:12], 'dx']
    Y_test.append(label)
Y_test = pd.get_dummies(Y_test)
Y_test = Y_test.values
print('Y_test:',Y_test.shape) # validate correct shape

Y_train: (27631, 7)
Y_val: (5262, 7)
Y_test: (1002, 7)


In [0]:
print(len(os.listdir('base_dir/train_dir/nv')))
print(len(os.listdir('base_dir/train_dir/mel')))
print(len(os.listdir('base_dir/train_dir/bkl')))
print(len(os.listdir('base_dir/train_dir/bcc')))
print(len(os.listdir('base_dir/train_dir/akiec')))
print(len(os.listdir('base_dir/train_dir/vasc')))
print(len(os.listdir('base_dir/train_dir/df')))

print(len(os.listdir('base_dir/val_dir/nv')))
print(len(os.listdir('base_dir/val_dir/mel')))
print(len(os.listdir('base_dir/val_dir/bkl')))
print(len(os.listdir('base_dir/val_dir/bcc')))
print(len(os.listdir('base_dir/val_dir/akiec')))
print(len(os.listdir('base_dir/val_dir/vasc')))
print(len(os.listdir('base_dir/val_dir/df')))

print(len(os.listdir('base_dir/test_dir/nv')))
print(len(os.listdir('base_dir/test_dir/mel')))
print(len(os.listdir('base_dir/test_dir/bkl')))
print(len(os.listdir('base_dir/test_dir/bcc')))
print(len(os.listdir('base_dir/test_dir/akiec')))
print(len(os.listdir('base_dir/test_dir/vasc')))
print(len(os.listdir('base_dir/test_dir/df')))

4004
3996
3997
4000
3949
4000
3685
804
777
784
800
753
699
645
657
136
111
49
25
12
12


In [0]:
np.save("../content/gdrive/My Drive/X_train_APP/NumpyFiles", X_train)
np.save("../content/gdrive/My Drive/Y_train_APP/NumpyFiles", Y_train)

np.save("../content/gdrive/My Drive/X_val_APP/NumpyFiles", X_val)
np.save("../content/gdrive/My Drive/Y_val_APP/NumpyFiles", Y_val)

np.save("../content/gdrive/My Drive/X_test_APP/NumpyFiles", X_test)
np.save("../content/gdrive/My Drive/Y_test_APP/NumpyFiles", Y_test)